# QTS Project  
## Multi-Factor Equity Selection + Leveraged Options Wheel Strategy

This project studies a systematic options wheel strategy implemented on listed American equity options. The framework integrates multi-factor stock selection with option premium harvesting under a margin-based capital structure.

The objective is to evaluate whether combining:

$$
\text{Equity Factor Premium} + \text{Volatility Risk Premium}
$$

can generate superior risk-adjusted returns under realistic leverage and transaction cost assumptions. 

---

**Course:** Quantitative Trading Strategy  
**Group:** Final Project Group PF: J 

**Group Members**

- **Name**:    **Student ID**: 
- **Name**:    **Student ID**: 
- **Name**: Theo Li          **Student ID**: 12503045
- **Name**: Jessica Xu       **Student ID**: 12503042
- **Name**: Catherine Chen   **Student ID**: 12496600

# 1. Project Overview

This project studies a systematic options wheel strategy implemented on listed American-style equity options written on U.S. large-cap stocks. The framework combines equity selection and option premium harvesting within a unified backtesting structure. The objective is to evaluate whether disciplined short-volatility exposure, when applied to a selected set of liquid underlying equities, can generate attractive risk-adjusted returns over an extended sample period.

The strategy operates through monthly rolling option positions while maintaining a multi-year investment horizon to ensure exposure across different market regimes. Both assignment and premium collection dynamics are explicitly modeled, allowing the portfolio to alternate between cash-secured puts and covered calls. Capital allocation, transaction costs, and execution frictions are incorporated to ensure realistic performance estimation.

The backtest spans more than five years and includes at least five distinct underlying equities, generating sufficient trade frequency for statistical evaluation. Performance is assessed across full-sample and stressed market environments in order to evaluate robustness under varying volatility conditions.

# 2. Data Loading, Cleaning, and Engineering Pipeline

This section defines the data infrastructure and preprocessing pipeline used to construct the research dataset. The objective is to transform raw equity and option data into a clean, aligned panel suitable for signal construction and backtesting.

The data sources are as follows:

- Daily listed option data from Databento API  
- Daily equity price data from Nasdaq Data Link  
- Daily stock fundamental data and analyst consensus data from WRDS / IBES  

All datasets are aligned to a common trading calendar and indexed by date and underlying ticker.

---

## 2.1 Equity Data Loading and Cleaning

This subsection loads daily equity price data from Nasdaq Data Link and merges it with fundamental and analyst consensus variables from WRDS / IBES. The dataset includes adjusted close prices, market capitalization, sector classification, and factor-related inputs.

The preprocessing procedure includes filtering the backtest period, removing securities with insufficient history, handling missing values, and constructing daily returns. Daily return is defined as:

$$
r_t = \frac{P_t}{P_{t-1}} - 1
$$

The output of this step is a cleaned equity panel indexed by date and ticker, which serves as the foundation for factor ranking and portfolio construction.

---

## 2.2 Option Data Loading and Cleaning

This subsection loads daily option data from Databento API. The raw dataset includes option price, strike, expiration date, implied volatility, option delta, and open interest.

Contracts are filtered to approximately 4-week maturity in order to match the monthly rolling design of the wheel strategy. Illiquid contracts are removed based on open interest thresholds. Option records are merged with underlying equity prices to ensure pricing consistency.

Realized volatility is computed from underlying returns, and the IV–RV spread is constructed as:

$$
IV - RV
$$

This spread serves as a proxy for the volatility risk premium embedded in option prices.

The output is an aligned option panel indexed by date, underlying ticker, and contract characteristics.

---

## 2.3 Data Snapshot and Diagnostic Validation

This subsection performs structural diagnostics to validate data integrity before signal construction. Time-series and cross-sectional visualizations are generated to examine implied volatility dynamics, IV percentile distribution, term structure patterns, skew behavior, and trade count coverage.

This step ensures that the dataset contains sufficient liquidity, consistent maturity selection, and stable coverage across multiple volatility regimes before proceeding to strategy implementation.

# 3. Strategy Architecture

This session defines the structural architecture of the strategy and clarifies how the equity selection layer interacts with the options execution layer. The objective is to establish a clear mapping between conceptual design and the implementation that follows in code.

The strategy consists of two interacting components:

1. A semiannual equity selection mechanism  
2. A monthly options wheel execution mechanism  

The equity layer determines the eligible stock universe at each rebalance date, while the option layer generates recurring premium income conditional on that selected universe.

Rebalance frequencies are defined as:

$$
T_{equity} = 6 \text{ months}
$$

$$
T_{option} = 4 \text{ weeks}
$$

The code in subsequent sections will construct these two timing cycles explicitly and implement their interaction within a unified backtesting engine.

# 4. Equity Selection Layer

This session constructs the equity selection mechanism that determines which underlying stocks are eligible for option writing. The objective is to formalize the stock ranking process and produce a time-indexed selected stock set.

The selection rule is based on a composite factor score defined as:

$$
Score_i = w_Q Q_i + w_M M_i + w_C C_i
$$

where $Q_i$, $M_i$, and $C_i$ denote standardized quality, momentum, and analyst consensus signals.

At each semiannual rebalance date, stocks are ranked cross-sectionally and the top-ranked securities are selected. The selected set is denoted as:

$$
\mathcal{S}_t = \{ i_1, i_2, ..., i_k \}
$$

The code in this session will compute factor signals, perform cross-sectional ranking, and construct the time-series of selected stock pools.

# 5. Option Signal Construction

This session defines the volatility-related signals that guide option selling decisions. The objective is to construct variables that capture the embedded volatility risk premium.

A key variable is the IV–RV spread:

$$
IV - RV
$$

where $IV$ denotes implied volatility and $RV$ denotes realized volatility computed from underlying returns.

Additional structural measures include implied volatility percentile, skew, and term structure slope. These variables provide information about volatility regime and tail risk pricing.

The code in this session will compute realized volatility, merge option-level implied volatility data, and construct the volatility signal panel used for trade filtering and regime diagnostics.

# 6. Entry Rules and Wheel Mechanics

This session formalizes the mechanical implementation of the options wheel strategy. The objective is to translate the conceptual wheel process into executable trade rules.

For each selected stock, a 4-week out-of-the-money put is sold. Strike selection is determined by delta targeting, typically under 10-delta and 20-delta regimes. Assignment probability is approximated by:

$$
P(\text{assignment}) \approx |\Delta|
$$

If the put expires in-the-money, the underlying stock is assigned. The strategy then transitions to a covered call position. If the call expires in-the-money, the stock is called away and the process returns to a cash-secured put phase.

The code in this session will simulate option expiration outcomes, handle assignment logic, and implement the state transition between cash, stock holding, and covered call positions.

# 7. Capital Allocation and Margin Model

This session defines the capital base and leverage framework under which the strategy operates. The objective is to ensure that position sizing and funding costs are explicitly modeled.

Initial capital is defined as:

$$
C_0 = 2{,}000{,}000
$$

Position sizes are determined subject to margin requirements. If a Reg-T framework is applied, position exposure must satisfy:

$$
Exposure \leq \frac{Capital}{Margin\ Requirement}
$$

If assignment results in stock ownership, borrowing costs are incorporated through a funding rate $r_{borrow}$.

The code in this session will compute margin-adjusted position sizes, update available capital dynamically, and incorporate funding costs into portfolio PnL.

# 8. Transaction Costs and Execution Modeling

This session incorporates realistic execution assumptions into the backtest. The objective is to prevent overestimation of strategy performance.

Option commissions are modeled as:

$$
0.3 \text{ USD per contract}
$$

Slippage is modeled as:

$$
P_{exec} = P_{mid}(1 \pm 0.01)
$$

where execution price deviates by 1% from mid-price.

The code in this session will adjust trade prices for commission and slippage and produce both gross and net performance series.

# 9. Backtest Engine

This session integrates all previous components into a unified portfolio simulation framework. The objective is to generate a daily time series of portfolio value.

Portfolio value is defined as:

$$
V_t = C_t + \sum Equity_t + \sum Option_t
$$

where $C_t$ denotes cash balance, and positions are marked to market daily.

The code in this session will iterate through time, execute monthly option rolls, update assignment states, adjust capital, and record daily portfolio value.

# 10. Performance Evaluation

This session evaluates the performance of the strategy over the full sample and selected subperiods. The objective is to quantify return, risk, and drawdown characteristics.

Key performance measures include:

- Annualized return  
- Annualized volatility  
- Sharpe ratio  
- Maximum drawdown  
- Value-at-Risk (VaR)  

The code in this session will compute these metrics for both gross and net returns and compare them across volatility regimes.

# 11. Portfolio-Level Risk Analysis

This session examines portfolio-level risk exposures beyond simple return metrics. The objective is to assess robustness under stress conditions.

Risk dimensions include market exposure, concentration risk, liquidity risk, and cross-asset correlation between underlyings.

Stress periods such as high-volatility regimes are examined to evaluate downside sensitivity.

The code in this session will compute correlation matrices, exposure measures, and stress-period performance statistics.

# 12. Benchmark Comparison

This session compares the strategy against passive alternatives. The objective is to determine whether excess returns are generated relative to standard benchmarks.

Excess return is defined as:

$$
\alpha = R_{strategy} - R_{benchmark}
$$

Benchmarks include buy-and-hold underlying exposure and broad market ETF performance.

The code in this session will compute benchmark returns and produce relative performance plots.